In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
import csv
# from api_key import api_key
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [2]:
export_url = (f"https://api.census.gov/data/timeseries/intltrade/exports")
import_url = (f"https://api.census.gov/data/timeseries/intltrade/imports")
china_code = "5700"
usa_code = "1000"
month = []
year = []

In [3]:
export_by_commodity = requests.get(f"{export_url}/hs?get=E_COMMODITY,E_COMMODITY_SDESC,ALL_VAL_MO,ALL_VAL_YR&time=from+2016-01&COMM_LVL=HS4&CTY_CODE={china_code}").json()


In [4]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4745:
    count += 1
    movalue.append(export_by_commodity[count][2])
    yrvalue.append(export_by_commodity[count][3])
    desc.append(export_by_commodity[count][1])
    period.append(export_by_commodity[count][4])
    code.append(export_by_commodity[count][0])
china_data_export = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [5]:
import_by_commodity = requests.get(f"{import_url}/hs?get=I_COMMODITY,I_COMMODITY_SDESC,GEN_VAL_MO,GEN_VAL_YR&time=from+2016-01&COMM_LVL=HS4&CTY_CODE={china_code}").json()


In [6]:
count = 1
movalue = []
yrvalue = []
period = []
desc = []
code = []
while count < 4802:
    count += 1
    movalue.append(import_by_commodity[count][2])
    yrvalue.append(import_by_commodity[count][3])
    desc.append(import_by_commodity[count][1])
    period.append(import_by_commodity[count][4])
    code.append(import_by_commodity[count][0])
china_data_import = pd.DataFrame({"YTD Value" : yrvalue,
                           "Monthly Value " : movalue,
                          "Period" : period,
                          "Commodity" : desc,
                          "Comm Code": code})

In [7]:
china_data_export["YTD Value"] =pd.to_numeric(china_data_export["YTD Value"])

In [8]:
first_2016 = china_data_export[china_data_export["Period"].str.contains("2016")]

In [9]:
data_2016 = first_2016.groupby(["Comm Code","Commodity"])["YTD Value"].sum()

In [10]:
test= pd.DataFrame({"total" : data_2016})

In [11]:
test

,,total
Comm Code,Commodity,
0101,"HORSES, ASSES, MULES AND HINNIES, LIVE",110695
0102,"BOVINE ANIMALS, LIVE",16500
0106,"ANIMALS, LIVE, NESOI",3994328
0203,"MEAT OF SWINE (PORK), FRESH, CHILLED OR FROZEN",392432378
0206,"ED OFFAL, BOVINE, SWINE, SHEEP, GOAT, HORSE, ETC.",150443229
0207,"MEAT & ED OFFAL OF POULTRY, FRESH, CHILL OR FROZEN",7436000
0209,PIG/POULTRY FAT FRSH CHLD FRZ SALT,3516050
0301,"FISH, LIVE",2840468
0302,"FISH, FRESH OR CHILLED (NO FILLETS OR OTHER MEAT)",719799


In [12]:
data_2016= test.nlargest(50,"total")


In [13]:
data_2016 = data_2016.reset_index()

In [14]:
data_2016= data_2016.nlargest(50,"total")
data_2016

,Comm Code,Commodity,total
0,1201,"SOYBEANS, WHETHER OR NOT BROKEN",12764555832
1,8800,"CIVILIAN AIRCRAFT, ENGINES, AND PARTS",11381074431
2,8703,MOTOR CARS & VEHICLES FOR TRANSPO,9284761335
3,8542,"ELECTRONIC INTEGRATED CIRCUITS & MICROASSEMBL,...",5670115603
4,8486,MACH/APPS FOR MANUFCT OF SEMIC,2999969295
5,8708,PARTS & ACCESS FOR MOTOR,2363044306
6,4707,WASTE AND SCRAP OF PAPER OR PAPERBOARD,2165407886
7,8517,ELECTRIC APPARATUS FOR LINE TEL,2052092393
8,4407,"WOOD SAWN OR CHIPPED LENGTH, SLICED ETC, OV6MM TH",1683832789
9,9018,"MEDICAL, SURGICAL, DENTAL",1620860589


In [15]:
data_2016 = data_2016.to_csv("data_2016.csv", index=False)